http://www.cs.sjtu.edu.cn/~jiangli/teaching/CS222/files/materials/Algorithm%20Design.pdf

## Chapter 1 : Stable Machine Problem (1-22)

Given a set of preferences among men and women, can we setup every men m and every women w with goal, that there, is a set of marriages with no instabilities. We’ll say that a matching S is stable if (i) it is perfect, and (ii) there is no instability with respect to S. Two questions spring immediately to mind:

* Does there exist a stable matching for every set of preference lists?
* Given a set of preference lists, can we efficiently construct a stable matching if there is one?


#### G-S (Gale-Shapely) Algorithm


1. Initially all $m \in M$ and $w \in W$ are free
2. While there is a man m who is free and hasn't proposed to every women, choose such a man m
3. Let w be the highest-ranked woman in m's prefrence list to whom m has not yet proposed
4. If w is free then
    (m, w) become engaged
5. Else w is currently engaged to m'
    If w prefers m' to m then,
        m remains free
    else w prefers m to m'
        (m, w) become engaged
        m becomes free
    Endif
6. Endif
7. EndWhile
8. Return the set S of engaged pairs


#### G-S algorithm terminates after at-most $n^2$ iterations of the While loop. 

#### Unfairness in G-S algorithm.

1. There could be multiple stable matching with the GS algorithm.
2. Example
            m prefers w to w'.
            m' prefers w' to w.
            w prefers m' to m.
            w' prefers m to m'.
3. Stable matchings: 
            (m,w), (m',w')
            (w,m'), (w',m)
4. In above example, we can see there is different outcome if men propose than women propose. And with more than 2 people on two sides, we can have much larger collection of solutions.
5. So above simple set of preference lists compactly summarizes a world in which someone is destined to end up unhappy: women are unhappy if men propose, and men are unhappy if women propose.

#### Proof for excercies: by contradiction

## Chapter 2 : Analysis of Algorithms (29-69)

* Generally we use worst-case-time to denote the running time of algorithm. However many alogrithm very rarely achieve worst-case. Another way is to use average-case but this will be very complicated to come up with random input cases. So we use worst-case-time.

#### Asymptotic Upper Bounds ( $ \mathcal{O} $ )
* Let T(n) be the worst-case running time of certain algorithm for input n.
* Given another function f(n), we say that T(n) = O(f(n)), **(read it as "T(n) is order f(n)")**
* For sufficiently large n, function T(n) is bounded above by a constant multiple of f(n).

> If there exists constants c>0 and n >= 0 so that for all n, we have T(n) <= c.f(n). T is asymptotically upperbound by f.

* If T(n) is O(n^2), it is correct to say that it is O(n^3). There are many possible upper bound of an algorithm.

#### Asymptotic Lower Bounds ( $ \omega $ )
* T(n) = Ω(f(n)) if and only if there exists constants c, n0 > 0 such that T(n) >= c.f(n) for all n > n0.

#### Asymptotic Tight Bounds ( $ \theta $ )
* Θ(f(n)) if only if there exists constants c1, c2, n0 > 0 such that c1.f(n) <= T(n) <= c2.f(n).


#### Common Running Time:

1. **O(n)** - Algorithms that traverse a list. **Example** - finding the max element, merge 2 sorted list
2. **O(logn)** - Algorithm that splits data into 2 parts and solve only one of the part recursively. **Example** - binary search
3. **O(nlogn)** - Algorithms that splits data into 2 parts and solve them independently. **Example** - mergesort.
4. **Quadratic O(n^2)** - Algorithm with pair of nested loop. **Example** - find closest pair of points among n points in space. 
           
           For every pair of point in space: (nC2)
               calucate distance
               
5. **Cubic O(n^3)** - Algorihtm with 3 nested loops. **Example** - Given sets S1, S2,..., Sn, each of which is a subset of n elements, and we would like to know whether some pair of these sets is disjoint. 

           For every pair of set (s1,s2): (nC2)
               For every point in s1: O(n)
                   check if point is in s2

6. **O(n^k)** - **Example** - Check independent set (set in which no 2 nodes are connected) of size k exists in graph or not.

            For every subset of k nodes in G: (nCk) - n^k / k!
                For every pair of nodes in subset: (kC2) - k^2 / 2
                    check if nodes are connected
            
            Total Running Time = (n^k * k^2)/2*k! = O(n^k) since k is constant

7. **O(2^n)** - Algorithms that consider all subset of graph. **Example** - Find an independent set of maximum size in graph
            
            For every subset in G: (2^n)
                For every pair of nodes in subset: (nC2)
                    check if nodes are connected
                    
            Total Running Time = 2^n * n^2/2 = O(n^2.2^n)
            
8. **O(n!)** - Matching n items with another n items or arrange n items in order. **Example** - Travelling salesman problem

## Chapter 3 : Graphs (73-107)

#### Terminology:
1. Connected Graph : In a graph for every pair of nodes there is a path between them. In directed it will be both ways.
2. Tree : If a graph is connected and there is no cycle.

#### BFS (Breadth First Search) :
* Uses **queue**, complexity : O(n+m).
* To determine connectivity, shortest path and minimum spanning tree of unweighted graph, to detect cycles, etc.
* Algorithm
        1. Initialize a boolean array, say Visited to keep track of visited nodes.
        2. Put the start node in the queue, make Visited[start]=1
        3. Lets pop the node iteratively from queue, exit loop if queue empty
            * print or perform any operation on node
            * Visit all unvisited neighbor of that node, and put them in queue one by one, make Visited[neighbor]=1

#### DFS (Depth First Search):
* Uses **stack**, complexity : O(n+m)
* Solving puzzles such as mazes, detecting cycles, testing bi-partitie graph.
* Algorithm
        1. Initialize a boolean array say Visited, to keep track of visited nodes.
        2. Put start node in stack, set Visited[start] = 1
        3. Lets pop the node iteratively from stack, exit loop if stack empty
            * print or perform any operation on node.
            * Visit all unvisited neighbor of that node, put them in stack one by one, make Visited[neighbor] = 1
        
          DFS(u):
            Mark u as "Explored" and add u to R
            For each edge (u, v) incident to u
                If v is not marked "Explored" then 
                    Recursively invoke DFS(v)
                Endif
            Endfor
            
            
#### Representing Graphs:
* There are 2 ways represent Graph(V,E). 1. Adjacency Matrix, 2. Adjacency List. Use n = |V| and m = |E|.
* For dense graph, edge between every node then, $ m = {n \choose 2} \le n^2$.
* For sparse graph, $ m \ge n-1 $
* Adjacency Matrix
      * Works better if graph is dense.
      * Represent graph in form of nxn matrix where M[u][v]=1 if there is an edge, else it is 0. Space complexity: O(n^2).
      * For graph with m << n^2, it is possible to represent graph in much lesses space.
      * Getting all the nearby vertices of a node will take O(n) time in worst case.

* Adjacency List
      * Works better for sparse graphs
      * Adj list contain all nodes and with each node there is a list of adjacent nodes. Space Complexity: O(n+m).
        
#### Testing Bipartiteness: application of BFS
* A bipartite graph is one where nodes in graph can be partitioned into sets X and Y, where for all edges one end lies in X and another in Y. Can be thought as a coloring problem where every edge has one end colored blue and another red.
* If a graph has odd cycle, it cannot be bi-partite.
* **Algorithm** : Firstly we color node s as red, then all nodes in layer L1 as blue, then all nodes in layer L2 as red and so on coloring odd-numbered layers blue and even-numbered layers red. In the end we will simple scan all edges if any edge has same color on both sides.

#### Strongly connected graph:
* A directed graph is strongly connected if every pair of nodes in the the graph is mutually reachable from one another.
* If u and v are mutually reachable, and v and w are mutually reachable, then u and w are mutually reachable
* For any two nodes s and t in a directed graph, their strong components are either identical or disjoint. (Proof - 99)
* **Algorithm to test** : run a bfs from s in G and again run a bfs from s in Grev. If in any of them it fails to reach every node, then graph is not strongly connected.


#### Directed-Acyclic-Graph (DAG) and topological sorting:
* A DAG is a directed graph with no cycles in it. Very common in CS to represent precedence relations.
* In DAG, there will always be a node with no incoming edges.
* **Algorithm for topological sorting**
        We Will maintain 2 data structures
        a) for each node w, the number of incoming edges w has from active nodes (nodes that not have been deleted).
        b) the set S of all active node in G that has no incoming edges from other active nodes
        
        1. At start all nodes are active, do a linear pass over all edges and initialize above two data structures.
        2. For each iteration select a node v from S and delete it and print it.
        3. After deleting v, subtract 1 from number of incoming edges of node u that has an edge from v.
        4. Go to step 2
        
#### Excercise:
    1. Excercise 5 : (Pg. 108)
    2. Excercise 6 : (Pg. 108)
    3. Excercise 8 : (pg. 109)
    
#### Proofs
    1. Contradiction
    2. Induction

## Chapter 4 : Greedy Algorithms (115-)

#### Introduction:
1. A greedy algorithm builds up a solution in small steps, choosing a decision at each step myopically to optimize some underlying criterion.
2. It is easy to invent greedy algorithms for almost any problem; finding cases in which they work well, and proving that they work well, is the interesting challenge.
3. To Proof:
    * establish that the greedy algorithm stays ahead, one sees that it does better than any other algorithm at each step; it then follows that it produces an optimal solution.
    * second is exchange argument, one consider any possible solution to problem and gradually transforms it into the solution found by greddy algorithm without hurting its quality.
    
#### Interal Scheduling
**Problem :** We have a set of request {1, 2, ..., n}, the $i^{th}$ request corresponds to an interval starting at $s_i$ and finishing at $f_i$. A subset of request is compatible if there is no overlap in any 2 of them. Goal is to accept as large a compatible subset as possible.

**Lets think of some greedy solutions :**
1. Most obvious would be to select the available request at the earlist, the **one with minimal $s_i$** so we start using resource as quickly as possible. Since our goal is statify as many request as possible. If first request is big, it would not be optimal.

2. We could start out by accepting the request that **requires the smallest interval of time $ f_i - s_i $**. Better but sub-optimal. A smaller request can prevent 2 bigger request from being satisfied (Pg. 117 (4.1.b)).

3. We could design a greedy algorithm, that is based on idea: for each request, we count the number of other request that are not compatible. We will **accept the request that has fewer number of noncompatible requests**. This is quite optimal but still not fully optimal in some cases (Pg. 117 (4.1.c))

4. A greedy rule that does lead to optimal solution: **we should accept first the request that finishes first**, that is the request i for which $f_i$ is as small as possible. **Proof by Induction (Pg. 120 4.2)**

#### Minimizing Lateness
**Problem** We again have a set of request {1, 2, .., n} with length ${(t_0, t_1, .. t_n)}$, and a single resource. Each request have a deadline ${(d_0, d_1,..., d_n)}$. Lateness is defined as $ L_i = f(i) - d_i $, Lateness is 0 if request i is not late. The goal is minimizing maximum lateness L.

**Lets think of some greedy solutions**
1. One approach is to schedule request in order of request length $t_i$, It completely ignores the deadline of jobs. Consider 2 requests with $(t_i, d_i)$ as (1,100), (10,10). The second job has to be started right away to make lateness 0.

2. From previous example, lets use slack time $d_i - t_i$ as ordering criteria. Unfortunately, the greedy rule fails here as well. Consider (1, 2) and (10,10). Here our approach would place second job first, first job will incur lateness 9. If it is otherwise second job will incur lateness of only 1.

3. There is however equally basic and optimal soultion i.e. sort jobs in increasing order of deadlines $d_i$ (Earliest deadline first). Its harder to believe this algorithm produces optimal solution (as it does not consider length). **Proof by exchange argument (Pg. 128 4.7 to 4.9)**

#### Optimal Caching
**Problem** Caching is a general term for process of storing a small data in fast memory so as to reduce amount of time spent interacting with slow memory. Initially cache holds some k items. A sequence of data items D = $d_1, d_2,..., d_m$ drawn from U (main memory) is presented to us.

When item $d_i$ is presented, we can access it very quickly if it is already in the cache; otherwise we are required to bring it from main memory into the cache and if cahce is full, to evict some other data that is currently in cache to make room for $d_i$. This is called cache miss, and we want to have few of these.

**Optimal Solution**
* We will call this the Farthest-in-Future Algorithm. When it is time to evict something, we look at the next time that each item in the cache will be referenced, and choose the one for which this is as late as possible.

* **Proof by exchange argument (Pg. 134 4.11)**

#### Shortest Path in a Graph (Dijiktras Algorithm)
* We start with starting node s and determines shortest path from s to other nodes in graph. The algorithm maintaines a set S of vertices u for which we have determined  a shortest path distance d(u) from s. this is explored part of the graph. Initially S={s} and d(s) = 0, For each node v $\in$ V-S we determine shortest path.
* Shortest path can be constructed by travelling along a path through the explored part S to some u $\in$ S, followed by single edge (u,v) where v $\in$ V-S. We consider quantity $d'(u) = d(u) + l_{(u,v)} $. We choose v for which d'(v) is minimized. Add that v to s and d(v) will be value of d'(v).
* **Proof : We will show it "stays ahead" of all other solution in each step. (Pg. 139 4.14) : Induction**

#### Minimum Spanning Tree
* Kruskals Algorithm - Union Find DS
* Prims Algorithm - Heap DS
* Reverse Delete Algorithm (Inverse of Kruskals)

#### Huffmann Encoding
* Given relative frequencies of each letter, give optimal encoding for each letter.
* **Solution :** Suppose that $y^*$ and $z^*$ are the lowest frequency letters in S. Now we will merge these two letters together and they end up as sibilings leaves below a common parent. Common parent acts like a "meta-letter" whose frequency is the sum of frequencies of $y*$ and $z*$. We recursively follow same steps until only single "meta-letter" remains.